<a href="https://colab.research.google.com/github/paulquimbo/nhanes_inferential_2021_23/blob/main/nhanes_inferential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyreadstat
!pip install pandas


In [2]:
import pyreadstat
import pandas as pd
import os

In [ ]:
#  checking for xpt file that are non readable with pyreadtat because of formatting issue
folder = "/content/raw/"
for file in os.listdir(folder):
    if file.endswith(".xpt"):
        path = os.path.join(folder, file)
        try:
            df, meta = pyreadstat.read_xport(path)
            print(f"✅ Loaded: {file}")
        except Exception as e:
            print(f"❌ Failed: {file} — {e}")

✅ Loaded: VID_L.xpt
✅ Loaded: HEPB_S_L.xpt
✅ Loaded: PAQ_L.xpt
❌ Failed: DEMO_L.xpt — 'utf-8' codec can't decode byte 0x92 in position 13: invalid start byte
✅ Loaded: KIQ_U_L.xpt
✅ Loaded: BPXO_L.xpt
✅ Loaded: WHQ_L.xpt


In [ ]:
df, meta = pyreadstat.read_xport(demo)
print(df.head())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 13: invalid start byte

In [ ]:
# Using R to convert xpt to csv
# Installing haven (only needed once)
install.packages("haven")

In [ ]:
# Load the library
library(haven)

# Set folder path
folder <- "/content/raw"

In [ ]:

# Converting all .xpt files to .csv

# List all .xpt files
xpt_files <- list.files(folder, pattern = "\\.xpt$", full.names = TRUE)

# Loop through and convert each to .csv
for (xpt_file in xpt_files) {
  # Read .xpt file
  data <- read_xpt(xpt_file)

  # Create .csv filename
  csv_file <- sub("\\.xpt$", ".csv", xpt_file)

  # Write to CSV
  write.csv(data, csv_file, row.names = FALSE)

  cat("✅ Converted:", xpt_file, "→", csv_file, "\n")
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



✅ Converted: /content/raw/BPXO_L.xpt → /content/raw/BPXO_L.csv 
✅ Converted: /content/raw/DEMO_L.xpt → /content/raw/DEMO_L.csv 
✅ Converted: /content/raw/HEPB_S_L.xpt → /content/raw/HEPB_S_L.csv 
✅ Converted: /content/raw/KIQ_U_L.xpt → /content/raw/KIQ_U_L.csv 
✅ Converted: /content/raw/PAQ_L.xpt → /content/raw/PAQ_L.csv 
✅ Converted: /content/raw/VID_L.xpt → /content/raw/VID_L.csv 
✅ Converted: /content/raw/WHQ_L.xpt → /content/raw/WHQ_L.csv 


In [ ]:
# converting single .xpt file to .csv
library(haven)
data <- read_xpt("/content/raw/DEMO_L.xpt")
write.csv(data, "/content/raw/DEMO_L.csv", row.names = FALSE)

In [5]:
# Mapping urls
demo = "/content/raw/DEMO_L.csv"
bp = "/content/raw/BPXO_L.csv"
vitd = "/content/raw/VID_L.csv"
hepb = "/content/raw/HEPB_S_L.csv"
kidney = "/content/raw/KIQ_U_L.csv"
minbehavior = "/content/raw/PAQ_L.csv"
weight = "/content/raw/WHQ_L.csv"

In [6]:
demo1 = pd.read_csv(demo)
print(demo1.head())

     SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0  130378        12         2         1        43       NaN         5   
1  130379        12         2         1        66       NaN         3   
2  130380        12         2         2        44       NaN         2   
3  130381        12         2         2         5       NaN         5   
4  130382        12         2         1         2       NaN         3   

   RIDRETH3  RIDEXMON  RIDEXAGM  ...  DMDHRGND  DMDHRAGZ  DMDHREDZ  DMDHRMAZ  \
0         6       2.0       NaN  ...       NaN       NaN       NaN       NaN   
1         3       2.0       NaN  ...       NaN       NaN       NaN       NaN   
2         2       1.0       NaN  ...       NaN       NaN       NaN       NaN   
3         7       1.0      71.0  ...       2.0       2.0       2.0       3.0   
4         3       2.0      34.0  ...       2.0       2.0       3.0       1.0   

   DMDHSEDZ      WTINT2YR      WTMEC2YR  SDMVSTRA  SDMVPSU  INDFMPIR  
0       N

In [7]:
#removing duplicates
demo1 = demo1.drop_duplicates()
